In [ ]:
# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import json
import datetime
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

import time


#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")

year_from = 2010
month_from = 1
year_to = 2019
month_to = 12

filename = str(year_from) + "_" + str(month_from) + "to" +  str(year_to) + "_" + str(month_to) + ".csv"

year_months = []

for i in range(year_from,year_to+1):
    for j in range(month_from,month_to+1):
        temp = [i, j] 
        year_months.append(temp)
        
        
        
# Oandaからcandleデータを取得する。
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    #adjust = datetime.timedelta(hours=-4)
    #adjust = datetime.timedelta(hours=-13)
    
    data = []
    for res in JSONRes['candles']:
        temp = [datetime.datetime.fromisoformat(res['time'][:19])]# + adjust] #indexとなる時間を追加
        
        temp.extend([datetime.datetime.fromisoformat(res['time'][:19]).hour]) #時間を追加
        
        month = datetime.datetime.fromisoformat(res['time'][:19]).month
        
        for i in range(1,13):
           
            if(month==i):
                temp.extend([1]) #月をonehotで追加
                #time.sleep(0.1)
            else:
                temp.extend([0])
                
        temp.extend([datetime.datetime.fromisoformat(res['time'][:19]).day])  #日を追加
        
        weekday = datetime.datetime.fromisoformat(res['time'][:19]).weekday
        
        for i in range(0,7):
            
            if(weekday==i):
                temp.extend([1]) #曜日をonehotで追加
                #time.sleep(0.00001)
            else:
                temp.extend([0])
        
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data

all_data = []
# year, monthでループ
#NY = datetime.timedelta(hours=-13) +  datetime.timedelta(minutes=-1)
for year, month in year_months:
    date_from = datetime.datetime(year, month, 1)# +NY
    date_to = date_from + relativedelta(months=+1, day=1)# +NY

    ret = getCandleDataFromOanda("USD_JPY", api, date_from, date_to, "M10")
    month_data = oandaJsonToPythonList(ret)#取得したデータを格納

    all_data.extend(month_data)#データを追加


print(np.array(all_data).shape)
print(all_data[0])

#ここからall_dataに追加すればよい


# pandas DataFrameへ変換
df = pd.DataFrame(all_data)

print(df)

df.columns = ['Datetime',"hour_range",
              "January","February","March","April","May","June","July","August","September","October","November","December",
              "day",
              "Sunday","Monday","Thuesday","Wednesday","Thursday","Friday","Saturday",
              'Volume', 'Open', 'High', 'Low', 'Close']
df = df.set_index('Datetime')

#print(df)

#print(df.isnull().any())
#print(year_months)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

print(df.isnull().any())

#cvs への保存
df.to_csv(filename)